# 2. Spam

Reporta el porcentaje de correos que están etiquetados como spam y como no spam en el conjunto de datos.


In [ ]:
import pandas as pd
import numpy as np 
from matplotlib import pyplot as plt

df = pd.read_csv('http://turing.iimas.unam.mx/~gibranfp/cursos/aprendizaje_automatizado/data/spam.csv', delim_whitespace=' ', header=None)
data = df.to_numpy()

# prior values without *1000
data_d = data[data[:, -1] == 0] # No spam
no_spam = data_d[:, :-1]
p1 = ((len(no_spam)*100)/len(data))
print("El porcentaje correos no spam es: {} %".format(round(p1,5)))

data_i = data[data[:, -1] == 1] #Spam
spam = data_i[:, :-1]
p2 = ((len(spam)*100)/len(data))
print("El porcentaje correos spam es: {} %".format(round(p2,5)))

El porcentaje correos no spam es: 70.99768 %
El porcentaje correos spam es: 29.00232 %


Divide aleatoriamente el conjunto de datos en el 60 % para entrenamiento, el 20 % para validación
y el 20 % restante para prueba usando 0 como semilla para tu generador de números
aleatorios

In [ ]:
np.random.seed(0)

# shuffle the dataset
np.random.shuffle(data)
shape = data.shape[0]

#Split data 60, 20 y 20
shape_train, shape_test = int(shape*.6), int(shape*.2) # int convertion to avoid slice index error 

train_data, test_data, val_data = data[:shape_train,], data[shape_train:shape_train + shape_test,], data[shape_train + shape_test:,] #100% of the data

train_data.shape[0], test_data.shape[0], val_data.shape[0]

print("Splited data: {} (60%), {}(20%), {}(20%)".format(train_data.shape[0],test_data.shape[0],val_data.shape[0]))

#Spam and no spam selection
data_spam = train_data[train_data[:, -1] == 1]
data_no_spam = train_data[train_data[:, -1] == 0]

#remove last column for convenience
x_spam = data_spam[:, :-1]
x_no_spam = data_no_spam[:, :-1]

#x_spam.shape[0], x_no_spam.shape[0]

Splited data: 3103 (60%), 1034(20%), 1035(20%)


### Entrena 2 clasificadores bayesianos ingenuos con distintas distribuciones
##### Clasificador bayesiano ingenuo para distribución multinomial

#### 1. Multinomial
Estimamos la a priori \\
Asumimos que la clases se distribuyen como una Bernoulli donde la estimación del parámetro $\hat{q}_C$ se reduce al conteo de frecuencia de la clase en los datos:

$$ \hat{q}_C = \frac{N_C}{N}$$

In [ ]:
# previously calculadted on cell 1
q_spam, q_no_spam = 0.7099768, 0.2900232

###### Verosimilitud 
Asumimos que los atributos de distribuyen como multinomiales; El estimado de cada parámetro es la frecuencia de la palabra en la clase ignorando el termino de normalización:

$$\hat{q}(w_{t}|C) = \frac{n_C(w_t)}{\sum_{w}^{} n_C(w_t)}$$

In [ ]:
#Params estimation

#occurrences ffor spam
v_spam_w = x_spam.sum(axis=0) 

#occurrences for No spam
v_nospam_w = x_no_spam.sum(axis=0) 

param_spam = v_spam_w / v_spam_w.sum()
param_no_spam = v_nospam_w / v_nospam_w.sum()

print(param_spam,param_no_spam)

[1.33238866e-02 6.49465667e-03 1.67286611e-03 ... 9.84038889e-05
 1.57446222e-04 2.36169333e-04] [1.81731985e-03 2.25754018e-05 0.00000000e+00 ... 2.25754018e-05
 2.25754018e-05 2.25754018e-05]


##### Clasificación

Recordemos que para clasificar en el modelo bayseiano ingenuo debemos computar:

$$ P(n|s) = \prod_{t=1}^{|V|} q(w_{t}|d)^{x_t} P (d)$$

$$ P(n|i) = \prod_{t=1}^{|V|} q(w_{t}|i)^{x_t} P (i)$$

$$C = \operatorname*{max} \bigg\{ P(n | d), P(n | i) \bigg\} $$

Usando el logaritmo por conveniencia de los datos:
$$ P(n|s) = \prod_{t=1}^{|V|} q(w_{t}|s)^{x_t} P (s) \propto \sum _{t=1}^{|V|} x_t \log q(w_{t}|s) + P(s)  $$

Creando una clase para predecir las clases de nuestros datos:

In [ ]:
"""
Mi compañera sofia me oriento en esta parte, ella encontro una solucion (ya que algunos 
parametros daban 0), sumandole una variable aux para que el log no tienda a ∞ 
"""
class MultinomialBN():
  
  aux =0.0000000000001
  binary_classes =[1, 0]
  classes = ['spam','no_spam']

  def predict_spam(self,x) :
    return (np.log(self.aux+param_spam) * x).sum() + np.log(q_spam)

  def predict_no_spam(self,x) : 
    return (np.log(self.aux+param_no_spam) * x).sum() + np.log(q_no_spam)

  def classify(self,x):
    return int(self.binary_classes[np.argmax([self.predict_spam(x), self.predict_no_spam(x)])])
  

Resultados

In [ ]:

if __name__ == "__main__":

  classifier = MultinomialBN()
  train_loop = []
  val_loop = []
  test_loop = []

  #vectors with our predictions
  for i in train_data[:,:-1]:
    train_loop.append(classifier.classify(i))

  for j in val_data[:,:-1]:
    val_loop.append(classifier.classify(j))

  for k in test_data[:,:-1]:
    test_loop.append(classifier.classify(k))

  #Compare my predictions with subsets
  train_success = np.mean((train_loop == train_data[:,-1]))
  validation_success = np.mean(val_loop == val_data[:,-1])
  test_sucess = np.mean(test_loop == test_data[:,-1])

  print('Exito en el conjunto de entremaniento: {}%'.format((train_success*100).round(3)))
  print('Exito en el conjunto de validación: {}%'.format((validation_success*100).round(3)))
  print('Exito en el conjunto de prueba: {}%'.format((test_sucess*100).round(3)))

Exito en el conjunto de entremaniento: 96.777%
Exito en el conjunto de validación: 97.198%
Exito en el conjunto de prueba: 96.809%


# Spam (Bernoulli)

##### Clasificación
Para el segundo caso utilizaremos una debernouli, en donde todo es igual hasta los valores de la apriori. 

Suponemos los aributos con distribución de Bernoulli y stimamos las ocurrencias de la palabra en cada clase:

$$\hat{q}(w_{t}|C) = \frac{n_C(w_t)}{\sum_{w}^{} n_C(w_t)}$$

In [ ]:
#occurrences ffor spam
spam_w = x_spam.sum(axis=0) 

#occurrences for No spam
nospam_w = x_no_spam.sum(axis=0) 

param_spam_b = spam_w / spam_w.sum()
param_no_spam_b = nospam_w / nospam_w.sum()

print(param_spam_b,param_no_spam_b)

[0.0113622  0.00533328 0.00183397 ... 0.0001054  0.00018972 0.00027404] [1.65855224e-03 0.00000000e+00 0.00000000e+00 ... 2.14006741e-05
 1.07003371e-05 3.21010112e-05]


Utilizando nuevamente el logaritmo 
$$ P(x|C) \propto \sum_{t=1}^{|V|} x_t \ln q + \sum_{t=1}^{|V|}(1-x_t) \ln (1-q) + \ln P(C)$$


$$C = \operatorname*{max} \bigg\{ P(n | s), P(n | noSpam) \bigg\} $$

In [ ]:
class BernoulliBN():
  
  aux =0.0000000000001
  binary_classes =[1, 0]
  classes = ['spam','no_spam']

  def predict_spam(self,x) :
    return (np.log(self.aux+param_spam) * x).sum() + (np.log(1 - (self.aux + param_spam_b)) * (1-x)).sum() + np.log(q_spam)

  def predict_no_spam(self,x) : 
    return (np.log(self.aux+param_no_spam) * x).sum() + (np.log(1 - (self.aux + param_no_spam_b)) * (1-x)).sum() + np.log(q_no_spam)

  def classify(self,x):
    return int(self.binary_classes[np.argmax([self.predict_spam(x), self.predict_no_spam(x)])])
  
  def train_function():

In [ ]:

if __name__ == "__main__":

  classifier = BernoulliBN()
  train_loop = []
  val_loop = []
  test_loop = []

  #vectors with our predictions
  for i in train_data[:,:-1]:
    train_loop.append(classifier.classify(i))

  for j in val_data[:,:-1]:
    val_loop.append(classifier.classify(j))

  for k in test_data[:,:-1]:
    test_loop.append(classifier.classify(k))

  #Compare my predictions with subsets
  train_success = np.mean((train_loop == train_data[:,-1]))
  validation_success = np.mean(val_loop == val_data[:,-1])
  test_sucess = np.mean(test_loop == test_data[:,-1])

  print('Exito en el conjunto de entremaniento: {}%'.format((train_success*100).round(3)))
  print('Exito en el conjunto de validación: {}%'.format((validation_success*100).round(3)))
  print('Exito en el conjunto de prueba: {}%'.format((test_sucess*100).round(3)))

Exito en el conjunto de entremaniento: 96.971%
Exito en el conjunto de validación: 96.329%
Exito en el conjunto de prueba: 97.099%
